In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [40]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
oil=pd.read_csv('oil.csv')
stores=pd.read_csv('stores.csv')
holiday=pd.read_csv('holidays_events.csv')
trans=pd.read_csv('transactions.csv')

In [41]:
display(df.head())
display(oil.head())
display(stores.head())
display(holiday.head())
display(trans.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [42]:
a=[]
for i in stores.state.unique():
    if i in holiday.locale_name.unique():
        a.append(i)
a



['Santo Domingo de los Tsachilas',
 'Cotopaxi',
 'Imbabura',
 'Santa Elena',
 'Loja',
 'Esmeraldas']

In [43]:
df1=pd.merge(df,stores,how='left')
df3=pd.merge(df_test,stores,how='left')

In [44]:
df1=pd.merge(df1,oil,how='left')
df3=pd.merge(df3,oil,how='left')

In [45]:
df2=pd.merge(df1,trans,how='left',on=['date','store_nbr'])
df3=pd.merge(df3,trans,how='left',on=['date','store_nbr'])

In [46]:
df2.fillna(0,inplace=True)
df3.fillna(0,inplace=True)

In [47]:
df2.head()
#import seaborn as sns
#sns.pairplot(df2)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,0.0


In [48]:
le=LabelEncoder()
df2['familynum']=le.fit_transform(df2.family)
df3['familynum']=le.fit_transform(df3.family)

In [49]:
df2.head()
x=df2[['store_nbr','onpromotion','cluster','dcoilwtico','transactions','familynum']]
y=df2[['sales']]


In [50]:

'''
import matplotlib.pyplot as plt

for g,i in enumerate(x.columns):
    plt.hist(x[i],bins=50)
    plt.xlabel(i)
    plt.show()
    x[[i]].boxplot()
    plt.show()
    '''

'\nimport matplotlib.pyplot as plt\n\nfor g,i in enumerate(x.columns):\n    plt.hist(x[i],bins=50)\n    plt.xlabel(i)\n    plt.show()\n    x[[i]].boxplot()\n    plt.show()\n    '

In [51]:
#df2.corr()['sales']
import seaborn as sns
#sns.pairplot(df2)

In [52]:


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)
test=df3[['store_nbr','onpromotion','cluster','dcoilwtico','transactions','familynum']]

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
scores=[]
models=[GradientBoostingRegressor(random_state=0),RandomForestRegressor(random_state=0)]
for model in models:
    model.fit(x_train,y_train)
    sc_tr=model.score(x_train,y_train)
    sc_te=model.score(x_test,y_test)
    scores.append((sc_tr,sc_te))

C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [54]:
scores

[(0.7897130207060719, 0.7811868948455325),
 (0.9847853963281683, 0.9009651202904295)]

In [55]:
test['sales']=models[1].predict(test)
df_test['sales']=test['sales']
df_test[['id','sales']].to_csv('submission.csv',index=False)

C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
df_test.head()

,id,date,store_nbr,family,onpromotion,sales
0,3000888,2017-08-16,1,AUTOMOTIVE,0,3.129357
1,3000889,2017-08-16,1,BABY CARE,0,0.000000
2,3000890,2017-08-16,1,BEAUTY,2,19.040000
3,3000891,2017-08-16,1,BEVERAGES,20,4790.830000
4,3000892,2017-08-16,1,BOOKS,0,0.740000
